In [1]:
!pip install -q datasets transformers onnx onnxruntime huggingface_hub

     |████████████████████████████████| 346 kB 27.7 MB/s 
     |████████████████████████████████| 4.2 MB 57.6 MB/s 
     |████████████████████████████████| 12.8 MB 55.2 MB/s 
     |████████████████████████████████| 5.2 MB 69.7 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 1.1 MB 45.1 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 140 kB 62.3 MB/s 
     |████████████████████████████████| 212 kB 47.7 MB/s 
     |████████████████████████████████| 596 kB 56.0 MB/s 
     |████████████████████████████████| 127 kB 76.9 MB/s 
     |████████████████████████████████| 6.6 MB 43.5 MB/s 
     |████████████████████████████████| 271 kB 63.6 MB/s 
     |████████████████████████████████| 144 kB 71.6 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 112 kB 67.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the

In [2]:
# !git clone https://github.com/huggingface/transformers.git

In [3]:
import numpy as np

from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from datasets import load_dataset, load_metric

import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [4]:
dataset = load_dataset("ziq/depression_advice")

Using custom data configuration ziq--depression_advice-522b8ba6acc78fca


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/ziq___csv/ziq--depression_advice-522b8ba6acc78fca/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
dataset["train"][0]

{'text': "Don't think you can deal with depression on your own. Would you avoid medical help for a chronic physical disease? Talk to trusted friends and family, as well as seeking professional support."}

In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

In [7]:
tokenizer.add_special_tokens({'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'})

3

In [8]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

In [9]:
tokenizer(dataset["train"][0]["text"])

{'input_ids': [3987, 470, 892, 345, 460, 1730, 351, 8862, 319, 534, 898, 13, 10928, 345, 3368, 3315, 1037, 329, 257, 10726, 3518, 4369, 30, 12167, 284, 13467, 2460, 290, 1641, 11, 355, 880, 355, 6095, 4708, 1104, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

cols = dataset["train"].column_names
dataset_ = dataset.map(tokenize_function, batched=True, remove_columns=cols)
dataset_

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 81
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 21
    })
})

In [11]:
data_collator = DataCollatorForLanguageModeling(
  tokenizer=tokenizer, mlm=False,
)

In [12]:
dataset['train'], dataset_["train"]

(Dataset({
     features: ['text'],
     num_rows: 81
 }), Dataset({
     features: ['input_ids', 'attention_mask'],
     num_rows: 81
 }))

In [13]:
tokenizer

PreTrainedTokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<BOS>', 'eos_token': '<EOS>', 'unk_token': '<|endoftext|>', 'pad_token': '<PAD>'})

In [16]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [17]:
training_args = TrainingArguments(
    output_dir="./depression_suggestion", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=70, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    evaluation_strategy="epoch",
    push_to_hub=True
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset_["train"],
    eval_dataset=dataset_["test"],
    # prediction_loss_only=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/ziq/depression_suggestion into local empty directory.


In [18]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 81
  Num Epochs = 70
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 210


***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  Num examples = 21
  Batch size = 64
***** Running Evaluation *****
  N

TrainOutput(global_step=210, training_loss=16.9717529296875, metrics={'train_runtime': 65.2419, 'train_samples_per_second': 86.907, 'train_steps_per_second': 3.219, 'total_flos': 92597035991040.0, 'train_loss': 16.9717529296875, 'epoch': 70.0})

In [19]:
trainer.save_model()

Saving model checkpoint to ./depression_suggestion
Configuration saved in ./depression_suggestion/config.json
Model weights saved in ./depression_suggestion/pytorch_model.bin
Saving model checkpoint to ./depression_suggestion
Configuration saved in ./depression_suggestion/config.json
Model weights saved in ./depression_suggestion/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.34k/319M [00:00<?, ?B/s]

Upload file runs/Jun07_07-12-01_f6447e164777/events.out.tfevents.1654585937.f6447e164777.73.0:  15%|#4        …

Upload file runs/Jun07_07-12-01_f6447e164777/1654585937.9187386/events.out.tfevents.1654585937.f6447e164777.73…

Upload file training_args.bin: 100%|##########| 3.17k/3.17k [00:00<?, ?B/s]

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/ziq/depression_suggestion
   6f72590..3187b8e  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/ziq/depression_suggestion
   3187b8e..aa16c86  main -> main



In [32]:
from transformers import pipeline

pipe = pipeline('text-generation', model="./depression_suggestion", tokenizer=tokenizer)

loading configuration file ./depression_suggestion/config.json
Model config GPT2Config {
  "_name_or_path": "./depression_suggestion",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },

In [38]:
# prompt = "Today I believe we can finally"
# input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# # generate up to 30 tokens
# outputs = model.generate(input_ids, do_sample=False, max_length=30)
# tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [33]:
depression_suggestions = []

for i in range(5):
  text = pipe('Depression can be solved by')[0]["generated_text"]
  depression_suggestions.append(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [34]:
depression_suggestions

['Depression can be solved by addressing the symptoms of depression by taking a short walk straight into the park, walking, or doing something on a regular basis, or simply by doing a walk for a short walk each day. Research suggests depression can be helpful',
 'Depression can be solved by listening to music, finding music, being active in the gym, doing volunteer work, participating in exercise, doing more volunteer work and more self-care. Research suggests listening to simple, daily activities such and other music',
 'Depression can be solved by taking a more and more daily activity as an exercise. Taking a more, more calmer, or less more structured activity can be helpful in reducing depression. However, it also can be a powerful way to increase both',
 'Depression can be solved by using self-help or therapy, using medication or other prescribed drugs. Research shows depression can take many different forms and strengthen people’s mental health. Findings point to a more effective 

# After several trial of exporting the model, we still unable to convert the pytorch huggingface model to onnx.

We are planning to use the model right in our web application and generate depression suggestion right in our web, but the convertion of onnx seems to doesn't suport gpt2 onnx convert. So we're going to generate a lot of suggestions to randomize in our web application.

## Let's generate thousands and thousands of unique suggestions.

In [35]:
def generate(num):
  depression_suggestions = []

  for i in range(num):
    text = pipe('Depression can be solved by')[0]["generated_text"]
    depression_suggestions.append(text)
  
  return depression_suggestions

In [36]:
generated_suggestions = generate(1000)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [38]:
generated_suggestions

['Depression can be solved by addressing the symptoms by getting yourself into a more positive mood through activities that strengthen your mood. You may find it helpful to engage in mindfulness research, particularly when you’re depressed. The best way to deal with stress',
 'Depression can be solved by helping to talk, talk, or engage in conversation. Research suggests listening to those same people helps to reduce depression symptoms. Research shows listening to people who talk are more likely to become depressed than those who do not. Research',
 'Depression can be solved by taking a daily routine or task group activity and taking a few minutes to rest. Keeping up with your routine can improve mental health and reduce symptoms of depression. Research shows that depression does not need to be fixed, it can',
 'Depression can be solved by adopting a simple change of routine. For example, when working on a schedule, feel free to lean on your back and lean on your back as long as you’r

In [37]:
import pandas as pd

dict = { "generated_suggestions": generated_suggestions }
df = pd.DataFrame(dict)
df.head()

generated_suggestions
0  Depression can be solved by addressing the sym...
1  Depression can be solved by helping to talk, t...
2  Depression can be solved by taking a daily rou...
3  Depression can be solved by adopting a simple ...
4  Depression can be solved by recognizing positi...

In [39]:
df.to_csv("./generated_suggestions.csv")

## Error Exporting Model

Export PyTorch model to ONNX format for serving with ONNX Runtime Web 

In [23]:
!pip install -q transformers[onnx]

     |████████████████████████████████| 440 kB 18.1 MB/s 
     |████████████████████████████████| 212 kB 76.2 MB/s 
     |████████████████████████████████| 78 kB 8.3 MB/s 
     |████████████████████████████████| 46 kB 4.3 MB/s 
     |████████████████████████████████| 55 kB 4.5 MB/s 
     |████████████████████████████████| 99 kB 11.8 MB/s 
     |████████████████████████████████| 86 kB 7.5 MB/s 


In [24]:
import transformers
import transformers.convert_graph_to_onnx as onnx_convert
from pathlib import Path

In [29]:
tokenizer = AutoTokenizer.from_pretrained("")
model = AutoModelForCausalLM.from_pretrained("ziq/depression_suggestion")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/ziq/depression_suggestion/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/c1e4657a3fe6a9a9f514b96037b2bee6f906b286f4b54bf168748e7b037a0c43.845ce83a708b0e2e2749b74a773656875e1544ab5b1f57c7527774769302caf2
Model config GPT2Config {
  "_name_or_path": "ziq/depression_suggestion",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_

OSError: ignored

In [ ]:
pipeline = transformers.pipeline("text-classification",model=model,tokenizer=tokenizer)

In [25]:
pipe("I am depressed")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I am depressed. I often feel the pressure of depression that separates me and the world. I often feel the pressure of depression because I’m weak. I often feel the pressure of my depression on my body, but being around other people will'}]

In [27]:
onnx_convert.convert_pytorch(pipe, opset=11, output=Path("depression_suggestion.onnx"), use_external_format=False)

Using framework PyTorch: 1.11.0+cu113
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Found output output_1 with shape: {0: 'batch', 2: 'sequence'}
Found output output_1 with shape: {0: 'batch', 2: 'sequence'}
Found output output_2 with shape: {0: 'batch', 2: 'sequence'}
Found output output_2 with shape: {0: 'batch', 2: 'sequence'}
Found output output_3 with shape: {0: 'batch', 2: 'sequence'}
Found output output_3 with shape: {0: 'batch', 2: 'sequence'}
Found output output_4 with shape: {0: 'batch', 2: 'sequence'}
Found output output_4 with shape: {0: 'batch', 2: 'sequence'}
Found output output_5 with shape: {0: 'batch', 2: 'sequence'}
Found output output_5 with shape: {0: 'batch', 2: 'sequence'}
Found output output_6 with shape: {0: 'batch', 2: 'sequence'}
Found output output_6 with shape: {0: 'batch', 2: 'sequence'}
Ensuring inputs are in corr

/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:1327: UserWarning: Provided key attention_mask for dynamic axes is not a valid input/output name
  warnings.warn("Provided key {} for dynamic axes is not a valid input/output name".format(key))
/usr/local/lib/python3.7/dist-packages/torch/onnx/utils.py:1330: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input output_1
  "Automatically generated names will be applied to each dynamic axes of input {}".format(key))


ValueError: ignored

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType
quantize_dynamic("depression_suggestion.onnx", "depression_suggestion-int8.onnx", 
                 weight_type=QuantType.QUInt8)

Ignore MatMul due to non constant B: /[MatMul_68]
Ignore MatMul due to non constant B: /[MatMul_73]
Ignore MatMul due to non constant B: /[MatMul_162]
Ignore MatMul due to non constant B: /[MatMul_167]
Ignore MatMul due to non constant B: /[MatMul_256]
Ignore MatMul due to non constant B: /[MatMul_261]
Ignore MatMul due to non constant B: /[MatMul_350]
Ignore MatMul due to non constant B: /[MatMul_355]
Ignore MatMul due to non constant B: /[MatMul_444]
Ignore MatMul due to non constant B: /[MatMul_449]
Ignore MatMul due to non constant B: /[MatMul_538]
Ignore MatMul due to non constant B: /[MatMul_543]


In [ ]:
from google.colab import files

In [ ]:
files.download("depression_suggestion-int8.onnx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# model = model.to("cpu")

In [ ]:
!apt-get install git-lfs

In [ ]:
# token="token"

In [ ]:
# model.push_to_hub("xtremedistil-l6-h384-go-emotion", use_auth_token=token)

In [ ]:
# tokenizer.push_to_hub("xtremedistil-l6-h384-go-emotion", use_auth_token=token)